In [2]:
using JuMP, NamedArrays, Clp

availability =
  [ 0 0 1 1 0 0 0 1 1 0 0 0 0
    0 1 1 0 0 0 0 0 1 1 0 0 0
    0 0 0 1 1 0 1 1 0 1 1 1 1
    0 0 0 1 1 1 1 1 1 1 1 1 0
    0 0 0 0 0 0 1 1 1 0 0 0 0
    0 1 1 0 0 0 0 0 1 1 0 0 0
    0 0 0 1 1 1 1 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0 1 1 1
    1 1 1 0 0 0 0 0 0 1 1 0 0
    0 0 0 0 0 0 0 1 1 0 0 0 0
    0 0 0 0 0 0 1 1 1 0 0 0 0
    1 1 0 0 0 1 1 1 1 0 0 1 1
    1 1 1 0 1 1 0 0 0 0 0 1 1
    0 1 1 1 0 0 0 0 0 0 0 0 0
    1 1 0 0 1 1 0 0 0 0 0 0 0 ]

TIMES = ["10:00","10:20","10:40","11:00","11:20","11:40","lunch","1:00","1:20","1:40","2:00","2:20","2:40"]
NAMES = [:Manuel,:Luca,:Jule,:Michael,:Malte,:Chris,:Spyros,:Mirjam,:Matt,:Florian,:Josep,:Joel,:Tom,:Daniel,:Anne ]
times = NamedArray( availability, (NAMES,TIMES), ("NAME","TIME"))

15×13 Named Array{Int64,2}
NAME ╲ TIME │ 10:00  10:20  10:40  11:00  …   1:40   2:00   2:20   2:40
────────────┼──────────────────────────────────────────────────────────
:Manuel     │     0      0      1      1  …      0      0      0      0
:Luca       │     0      1      1      0         1      0      0      0
:Jule       │     0      0      0      1         1      1      1      1
:Michael    │     0      0      0      1         1      1      1      0
:Malte      │     0      0      0      0         0      0      0      0
:Chris      │     0      1      1      0         1      0      0      0
:Spyros     │     0      0      0      1         0      0      0      0
:Mirjam     │     1      1      0      0         0      1      1      1
:Matt       │     1      1      1      0         1      1      0      0
:Florian    │     0      0      0      0         0      0      0      0
:Josep      │     0      0      0      0         0      0      0      0
:Joel       │     1      1      0    

In [19]:
m = Model(solver = ClpSolver())

@variable(m, x[TIMES,NAMES] >= 0)
#print(getvalue(x["10:00",:Manuel]))

# each senior employee only interviews once
@constraint(m, a[j in NAMES], sum(x[i,j] for i in TIMES) == 1 )

# each time period has one interviewer except lunch time
# lunch time period can have three interviews
@constraint(m, b[i in TIMES[1:6]], sum(x[i,j] for j in NAMES) == 1 )
@constraint(m, c["lunch"], sum(x["lunch",j] for j in NAMES) == 3 )
@constraint(m, d[i in TIMES[8:13]], sum(x[i,j] for j in NAMES) == 1 )

# need to be fixed
@objective(m, Max, sum( x[i,j]*times[j,i] for i in TIMES, j in NAMES ) )

solve(m)

assignment = NamedArray( [ (getvalue(x[i,j])) for i in TIMES, j in NAMES ], (TIMES, NAMES), ("TIME","NAME"))

for i in TIMES
    for j in NAMES
        if(assignment[i,j] != 0)
            println(i," ",j)
        end
    end
end

10:00 Mirjam
10:20 Luca
10:40 Manuel
11:00 Daniel
11:20 Jule
11:40 Anne
lunch Malte
lunch Spyros
lunch Joel
1:00 Florian
1:20 Josep
1:40 Chris
2:00 Matt
2:20 Michael
2:40 Tom
